In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
from cts_recommender import logging_setup
logging_setup.setup_logging('INFO')

In [8]:
from cts_recommender.preprocessing import X_ML_feature_processing
from cts_recommender.settings import get_settings

cfg = get_settings()

processed_df, enriched_df = X_ML_feature_processing.load_processed_and_enriched_programming(
    data_path_processed=cfg.processed_dir / "programming.parquet", 
    data_path_enriched=cfg.processed_dir / "programming_enriched.parquet"
    )

full_df = X_ML_feature_processing.build_X_features(processed_df, enriched_df)
full_df.head()

2025-09-24 16:50:29 INFO cts_recommender.preprocessing.X_ML_feature_processing: Index(['date', 'title', 'description', 'channel', 'start_time', 'class_key',
       'rt_m', 'pdm', 'hour', 'weekday', 'is_weekend', 'duration_min',
       'season', 'public_holiday'],
      dtype='object')
2025-09-24 16:50:29 INFO cts_recommender.preprocessing.X_ML_feature_processing: Index(['date', 'title', 'description', 'channel', 'start_time', 'class_key',
       'rt_m', 'pdm', 'hour', 'weekday', 'is_weekend', 'duration_min',
       'season', 'public_holiday'],
      dtype='object')


KeyError: 'adult'

In [ ]:
from cts_recommender.preprocessing import movie_features
from cts_recommender.settings import get_settings

cfg = get_settings()
df = movie_features.load_processed_programming(cfg.processed_dir / "programming.parquet")
enriched_df = movie_features.enrich_programming_with_movie_metadata(df)
enriched_df.head()

In [7]:
from cts_recommender.adapters.tmdb import tmdb
tmdb_api = tmdb.TMDB_API()
id = tmdb_api.find_best_match("Inception", 148)
type(id)

2025-09-24 14:52:41 DEBUG urllib3.connectionpool: Starting new HTTPS connection (1): api.themoviedb.org:443
2025-09-24 14:52:42 DEBUG urllib3.connectionpool: https://api.themoviedb.org:443 "GET /3/search/movie?query=Inception&include_adult=true&language=fr HTTP/1.1" 200 None
2025-09-24 14:52:42 DEBUG cts_recommender.adapters.tmdb.tmdb: Found 10 results for title 'Inception'
2025-09-24 14:52:42 DEBUG cts_recommender.adapters.tmdb.tmdb: length of results: 10


int